# V.P. de Sentencias

Reporte de Sentencias.

In [ ]:
from datetime import datetime

# Pandas, matplotlib, seaborn
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 12,4
import warnings
warnings.filterwarnings('ignore')

# SQLAlchemy
from plataforma_web.app import create_app
from plataforma_web.extensions import db
app = create_app()
db.app = app

# Plataforma Web
from plataforma_web.blueprints.distritos.models import Distrito
from plataforma_web.blueprints.autoridades.models import Autoridad
from plataforma_web.blueprints.sentencias.models import Sentencia

In [ ]:
# Distritos Judiciales
distritos_select = db.session.query(Distrito.id, Distrito.nombre).\
    filter(Distrito.es_distrito_judicial == True).\
    filter(Distrito.estatus == "A").statement
distritos = pd.read_sql_query(sql=distritos_select, con=db.engine)
distritos.head(8)

In [ ]:
# Distrito Judicial a estudiar
distrito = Distrito.query.get(8)  # 6: Saltillo, 8: Torreon
print(distrito.id, distrito.nombre)

In [ ]:
# Juzgados
autoridades_select = db.session.query(Autoridad.id, Autoridad.clave, Autoridad.descripcion).\
    filter(Autoridad.distrito_id == distrito.id).\
    filter(Autoridad.es_jurisdiccional == True).\
    filter(Autoridad.es_notaria == False).\
    filter(Autoridad.estatus == "A").statement
autoridades = pd.read_sql_query(sql=autoridades_select, con=db.engine)
autoridades.head(4)

In [ ]:
# Constantes
DESDE = datetime(year=2021, month=8, day=1)
HASTA = datetime(year=2021, month=8, day=31)

# Sentencias
sentencias_select = db.session.query(Sentencia.id, Autoridad.clave, Sentencia.archivo, Sentencia.creado).\
    join(Autoridad).\
    filter(Sentencia.autoridad_id.in_(autoridades.id)).\
    filter(Sentencia.creado >= DESDE).\
    filter(Sentencia.creado <= HASTA).\
    filter(Sentencia.estatus == "A").statement
sentencias = pd.read_sql_query(sql=sentencias_select, con=db.engine)
sentencias.head()

In [ ]:
# Total
len(sentencias)

In [ ]:
# Contar las sentencias entregadas por juzgado
sentencias.clave.value_counts()

In [ ]:
# Al juntar las sentencias por dia van a faltar los que no tengan
incompletos = sentencias.groupby(sentencias['creado'].dt.date).size().reset_index(name='cantidad')
incompletos.set_index(pd.DatetimeIndex(incompletos.creado), inplace=True)
#incompletos.head()

# Completar la serie de dias
diarios = incompletos.resample("D").asfreq(0)
diarios.creado = diarios.index.values
#diarios.head()

# Graficar las sentencias por dia
plt.bar(diarios.creado, diarios.cantidad)
plt.xticks(list(diarios.creado), list(diarios.creado), rotation='vertical')
plt.show()